In [1]:
import pandas as pd
import os
import numpy as np
import re

### Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
film = pd.read_csv('/Users/anastasia/DS_NET/12_pandas/ml-latest-small/movies.csv')
rating = pd.read_csv('/Users/anastasia/DS_NET/12_pandas/ml-latest-small/ratings.csv')

In [3]:
film.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [4]:
rating.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [5]:
def categorization(data):
    if data['rating'].mean()<=2:
        return 'Низкий рейтинг'
    elif 4.5 <=data['rating'].mean()<=5:
        return 'Высокий рейтинг'
    else:
        return 'Средний рейтинг'

In [6]:
smth = rating.groupby('movieId').apply(categorization).to_frame('class').reset_index()

In [7]:
film = pd.merge(smth, film, on = 'movieId')

In [8]:
film.head(4)

,movieId,class,title,genres
0,1,Средний рейтинг,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Средний рейтинг,Jumanji (1995),Adventure|Children|Fantasy
2,3,Средний рейтинг,Grumpier Old Men (1995),Comedy|Romance
3,4,Средний рейтинг,Waiting to Exhale (1995),Comedy|Drama|Romance


### Задание 2
Используем файл `keywords.csv`.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце `region` пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим `undefined`.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

`geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
             'Северо-Запад': ['петербург', 'псков', 'мурманск'],
             'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}`

Результат классификации запишите в отдельный столбец `region`.

In [9]:
key = pd.read_csv('/Users/anastasia/DS_NET/12_pandas/ml-latest-small/keywords.csv')

In [10]:
key.head(2)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309


In [11]:
def geo(word):
    geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
             'Северо-Запад': ['петербург', 'псков', 'мурманск'],
             'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}
    for k, v in geo_data.items():
        if word in v:
            return k
    return 'undefined'

In [12]:
key['region']  = key['keyword'].apply(geo)

In [13]:
key[key['keyword'] == 'псков']

,keyword,shows,region
11222,псков,24885,Северо-Запад


In [14]:
geo('псков')

'Северо-Запад'

### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [15]:
film_2 = pd.read_csv('movies.csv')
rating_2 = pd.read_csv('ratings.csv')

In [16]:
film_2.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [17]:
def production_year(string):
    try:
        return int(re.findall(r'[12]\d{3}', string)[-1])
    except IndexError:
        return 1900

In [18]:
production_year('djie')

1900

In [19]:
film_2['year'] = film_2['title'].apply(production_year)

In [20]:
film_2.sort_values('year').head(4)

,movieId,title,genres,year
9259,156605,Paterson,(no genres listed),1900
9179,149334,Nocturnal Animals,Drama|Thriller,1900
9525,171891,Generation Iron 2,(no genres listed),1900
9515,171631,Maria Bamford: Old Baby,(no genres listed),1900


In [23]:
smth2 = film_2[['year' ,'movieId']]

In [24]:
smth2

,year,movieId
0,1995,1
1,1995,2
2,1995,3
3,1995,4
4,1995,5
...,...,...
9737,2017,193581
9738,2017,193583
9739,2017,193585
9740,2018,193587


In [25]:
rating = pd.merge(smth2, rating, on = 'movieId')

In [44]:
df = rating.groupby('year').mean().reset_index()[['rating', 'year']].sort_values('rating', ascending = False)

In [45]:
df

,rating,year
6,4.437500,1921
1,4.333333,1902
13,4.250000,1928
4,4.250000,1917
8,4.250000,1923
...,...,...
85,3.358936,2000
98,3.345116,2013
101,3.263393,2016
2,3.000000,1915


In [46]:
df['rating'].corr(df['year'])

-0.46241219982730514